<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap11_fine_tuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install "datasets>=2.18.0,<3" transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

In [3]:
from datasets import load_dataset

# データセットを読み込み、学習用とテスト用に分割
tomatoes = load_dataset("rotten_tomatoes")
train_data, test_data = tomatoes["train"], tomatoes["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# モデルとトークナイザーを読み込む
model_id = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
  """入力テキストのトークン化"""
  return tokenizer(examples["text"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
  """F1スコアを計算"""
  logits, labels = eval_pred
  predictions = np.argmax(logits,axis=-1)
  load_f1 = evaluate.load("f1")
  f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
  return {"f1": f1}

In [7]:
from transformers import TrainingArguments, Trainer

# 学習用の引数
training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

# 学習プロセスを実行する Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-558874027.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.9502196907997131,
 'eval_model_preparation_time': 0.0424,
 'eval_f1': 0.0,
 'eval_runtime': 334.3774,
 'eval_samples_per_second': 3.188,
 'eval_steps_per_second': 0.2}

11.1.2

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
for name, param in model.named_parameters():
  print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [11]:
for name, param in model.named_parameters():
  # 分類ヘッドは学習可能のまま
  if name.startswith("classifier"):
    param.requires_grad = True
  # それ以外は凍結
  else:
    param.requires_grad = False

In [12]:
from transformers import TrainingArguments, Trainer

# 学習プロセスを実行するTrainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

/tmp/ipython-input-1817132314.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
# モデルを読み込む
model_id = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# エンコーダーブロック１１はインデックス１６５から始まるのでそのブロックより前のすべての層を凍結する
for index, (name, param) in enumerate(model.named_parameters()):
  if index < 165:
    param.requires_grad = False
  else:
    param.requires_grad = True

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

In [ ]:
trainer.evaluate()

11.2.2 FewShot

In [4]:
from setfit import sample_dataset

sampled_train_data = sample_dataset(tomatoes["train"], num_samples=16)

In [5]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [6]:
from setfit import TrainingArguments as SetFitTrainingArguments
from setfit import Trainer as SetFitTrainer

args = SetFitTrainingArguments(
    # 対照学習のエポック数
    num_epochs=3,
    # ペアを生成する反復回数
    num_iterations=20,
    report_to="none"
)
args.eval_strategy = args.evaluation_strategy

# Trainer を作成する
trainer = SetFitTrainer(
    model=model,
    args=args,
    train_dataset=sampled_train_data,
    eval_dataset=test_data,
    metric="f1"
)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

***** Running training *****
  Num unique pairs = 1280
  Batch size = 16
  Num epochs = 3
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

11.3 マスク言語モデリングによる継続事前学習

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# マスク言語モデリング用のモデルを読み込む
model = AutoModelForMaskedLM.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

# テキストをトークン化
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_train = tokenized_train.remove_columns("label")
tokenized_test = test_data.map(preprocess_function, batched=True)
tokenized_test = tokenized_test.remove_columns("label")

In [ ]:
from transformers import DataCollatorForLanguageModeling

# トークンのマスキング
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [ ]:
# 学習用の引数を定義する
training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

# Trainer を初期化する
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
# 事前学習済みトークナイザーを保存する
tokenizer.save_pretrained("mlm")

# モデルを学習する
trainer.train()

# 更新されたモデルを保存する
model.save_pretrained("mlm")

In [ ]:
from transformers import pipeline

# モデルを読み込んで予測をする
mask_filter = pipeline("fill-mask", model="bert-base-cased")
preds = mask_filter("What a horrible [MASK]!")

# 結果を表示する
for pred in preds:
  print(f">>> {pred["sequence"]}")

In [ ]:
# mlm モデルを読み込んで予測をする
mask_filter = pipeline("fill-mask", model="mlm")
preds = mask_filter("What a horrible [MASK]!")

# 結果を表示する
for pred in preds:
  print(f">>> {pred["sequence"]}")

In [ ]:
from transformers import AutoModelForSequenceClassification

# 分類用にファインチューニングする
model = AutoModelForSequenceClassification.from_pretrained("mlm", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("mlm")

In [ ]:
11.4 固有表現認識

In [7]:
# 固有表現認識用のCoNLL-2003データセットを読み込む
dataset = load_dataset("conll2003", trust_remote_code=True)

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [8]:
example = dataset["train"][848]
example

{'id': '848',
 'tokens': ['Dean',
  'Palmer',
  'hit',
  'his',
  '30th',
  'homer',
  'for',
  'the',
  'Rangers',
  '.'],
 'pos_tags': [22, 22, 38, 29, 16, 21, 15, 12, 23, 7],
 'chunk_tags': [11, 12, 21, 11, 12, 12, 13, 11, 12, 0],
 'ner_tags': [1, 2, 0, 0, 0, 0, 0, 0, 3, 0]}

In [9]:
label2id = {
    "0":0, "B-PER":1, "I-PER":2, "B-ORG":3, "I-ORG":4,
    "B-LOC":5, "I-LOC":6, "B-MISC":7, "I-MISC":8
}
id2label = {index: label for label, index in label2id.items()}
label2id

{'0': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [10]:
from transformers import AutoModelForTokenClassification

# トークナイザーを読み込む
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# モデルを読み込む
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# ここの単語をサブワード単位のトークンに分割する
token_ids = tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
sub_tokens = tokenizer.convert_ids_to_tokens(token_ids)
sub_tokens

['[CLS]',
 'Dean',
 'Palmer',
 'hit',
 'his',
 '30th',
 'home',
 '##r',
 'for',
 'the',
 'Rangers',
 '.',
 '[SEP]']

In [12]:
def align_labels(examples):
  token_ids = tokenizer(
      examples["tokens"],
      truncation=True,
      is_split_into_words=True
  )
  labels = examples["ner_tags"]

  updated_labels = []
  for index, label in enumerate(labels):

    # トークンをそれぞれの単語にマッピングする
    word_ids = token_ids.word_ids(batch_index=index)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:

      # 新しい単語の開始
      if word_idx != previous_word_idx:

        previous_word_idx = word_idx
        updated_label = -100 if word_idx is None else label[word_idx]
        label_ids.append(updated_label)

      # 特殊トークンは-100
      elif word_idx is None:
        label_ids.append(-100)

      # ラベルがB-XXXの場合、I-XXXに変更する
      else:
        updated_label = label[word_idx]
        if updated_label % 2 == 1:
          updated_label += 1
        label_ids.append(updated_label)
    updated_labels.append(label_ids)
  token_ids["labels"] = updated_labels
  return token_ids

tokenized = dataset.map(align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [13]:
# オリジナルと更新されたラベルの違い
print(f"Original: {example['ner_tags']}")
print(f"Updated: {tokenized['train'][848]['labels']}")

Original: [1, 2, 0, 0, 0, 0, 0, 0, 3, 0]
Updated: [-100, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, -100]


In [14]:
import evaluate

# seqevalを読み込む
seqeval = evaluate.load("seqeval")

def compute_metrics(eval_pred):
  # 予測を作成する
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=2)

  true_predictions =[]
  true_labels = []

  # 文書単位での反復処理
  for predixtion, label in zip(predictions, labels):

    # トークン単位での反復処理
    for token_prediction, token_label in zip(prediction, label):

      # 特殊トークンを無視する
      if token_label != -100:
        true_predictions.append([id2label[token_prediction]])
        true_labels.append([id2label[token_label]])

  results = seqeval.conpute(
      predictions=true_predictions,
      references=true_labels
  )
  return {"f1": results["overall_f1"]}

11.4.2

In [15]:
from transformers import DataCollatorForTokenClassification

# トークン分類のデータコレーター
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer, AutoModelForTokenClassification

# 引数
training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

# モデルを読み込む
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)


# Trainer を初期化する
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-52195137.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


In [ ]:
# テストデータセットを用いてモデルを評価する
trainer.evaluate()

In [ ]:
from transformers import pipeline

# ファインチューニングしたモデルを保存する
trainer.save_model("ner_model")

# ファインチューニングしたモデルで推論を実行する
token_classifier = pipeline(
    "token-classification",
    model="ner_model",
)
token_classifier("My name is Maarten.")